# Python Algorithmic Trading Cookbook

## Chapter 10: Algorithmic Trading: Paper Trading

This Jupyter Notebook is created using Python version 3.8.2

----

### Requirements

You can install the requirements for this Jupyter Notebook by executing the below cell

In [0]:
!pip install pyalgotrading

----

### Recipe 1: EMA-Regular-Order Strategy: Fetching the Strategy

In [1]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [2]:
algobulls_connection = AlgoBullsConnection()

In [3]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [4]:
algobulls_connection.set_access_token('80b7a69b168c5b3f15d56688841a8f2da5e2ab2c')

In [5]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

,strategyCode,strategyName
0,49287246f9704bbcbad76ade9e2091d9,EMA Regular Order Strategy
1,4faf514fe096432b8e9f80f5951bd2ea,MACD Bracket Order Strategy


In [6]:
strategy_code1 = all_strategies.iloc[0]['strategyCode']
strategy_code1

'49287246f9704bbcbad76ade9e2091d9'

In [7]:
strategy_details1 = algobulls_connection.get_strategy_details(strategy_code1)
print(strategy_details1)

class StrategyEMARegularOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.timeperiod1 = self.strategy_parameters['timeperiod1']
        self.timeperiod2 = self.strategy_parameters['timeperiod2']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'EMA Regular Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        ema_x = talib.EMA(hist_data['close'], timeperiod=self.timeperiod1)
        ema_y = talib.EMA(hist_data['close'], timeperiod=self.timeperiod2)
        crossover_value = self.utils.crossover(ema_x, ema_y)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = 

### Recipe 2: EMA-Regular-Order Strategy: Paper Trading the Strategy

In [8]:
from datetime import time
from pyalgotrading.constants import *

In [9]:
instruments = algobulls_connection.search_instrument('SBIN')
instruments

[{'id': 7, 'value': 'NSE:SBIN'}]

In [10]:
instrument = instruments[0]['value']
instrument

'NSE:SBIN'

In [11]:
algobulls_connection.papertrade(strategy_code=strategy_code1, 
                                start_time=time(hour=9, minute=15), 
                                end_time=time(hour=15, minute=30), 
                                instrument=instrument, 
                                lots=1,
                                strategy_parameters={
                                    'timeperiod1': 5,
                                    'timeperiod2': 12
                                }, 
                                candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting PAPERTRADING job... Success.


In [12]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTING'}

In [13]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STARTED'}

### There's more...

In [14]:
algobulls_connection.stop_papertrading_job(strategy_code1)

Stopping PAPERTRADING job... Success.


In [15]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STOPPING'}

In [16]:
algobulls_connection.get_papertrading_job_status(strategy_code1)

{'data': 'STOPPED'}

<img src="images/diagrams/abc-job-status-state-diagram.png" width='50%'>

### Recipe 3: EMA-Regular-Order Strategy: Fetching Paper Trading Logs in Realtime

In [17]:
logs = algobulls_connection.get_papertrading_logs(strategy_code1)
print(logs)

[2020-07-09 09:12:18] Logs not available yet. Please retry in sometime.


In [18]:
logs = algobulls_connection.get_papertrading_logs(strategy_code1)
print(logs)

[2020-07-09 09:12:31] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-09 09:12:31] ABBroker connection has been setup successfully.
[2020-07-09 09:12:31] Sanity checks on cfg successful.
[2020-07-09 09:12:31] Setting up broker connection...
[2020-07-09 09:12:31] Broker connection has been setup successfully.
[2020-07-09 09:12:31] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-09 09:12:31] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0)... 
########################################
[2020-07-09 09:12:31] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-09 09:12:31] Reading strategy...
[2020-07-09 09:12:31] Entering Paper Trading mode. Henceforth, all timestamps will be Paper Trading timestamps...
[PT] [2020-07-09 09:15:00] [INFO] [tls] STARTING ALGOBULLS CORE...
[PT] [2020-07-09 09:15:00] [INFO] [tls] 
            
  #####  #######    #    ####

### Recipe 4: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Profit-&-Loss Table

In [19]:
algobulls_connection.get_papertrading_report_pnl_table(strategy_code1)

,instrument,entry_timestamp,entry_transaction_type,entry_quantity,entry_price,exit_timestamp,exit_transaction_type,exit_quantity,exit_price,pnl_absolute,pnl_percentage,pnl_cumulative_absolute,pnl_cumulative_percentage
0,NSE_EQ:SBIN,2020-07-09 12:45:00,BUY,1,194.45,2020-07-09 15:30:00,SELL,1,200,5.55,2.85,3,1.53
1,NSE_EQ:SBIN,2020-07-09 11:30:00,SELL,1,194.05,2020-07-09 12:45:00,BUY,1,194.45,-0.4,-0.21,-2.55,-1.32
2,NSE_EQ:SBIN,2020-07-09 11:15:00,BUY,1,194.75,2020-07-09 11:30:00,SELL,1,194.05,-0.7,-0.36,-2.15,-1.11
3,NSE_EQ:SBIN,2020-07-09 10:45:00,SELL,1,194,2020-07-09 11:15:00,BUY,1,194.75,-0.75,-0.39,-1.45,-0.75
4,NSE_EQ:SBIN,2020-07-09 10:30:00,BUY,1,194.7,2020-07-09 10:45:00,SELL,1,194,-0.7,-0.36,-0.7,-0.36


### Recipe 5: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Statistics Table

In [20]:
algobulls_connection.get_papertrading_report_statistics(strategy_code1)

,highlight_type,highlight_value
0,Net PnL,3.00
1,Net PnL %,1.53
2,Max Drawdown,-2.55
3,Max Drawdown %,-1.31
4,Number of Trades,5.00
5,Number of Wins,1.00
6,Number of Looses,4.00
7,Number of Long Trades,3.00
8,Number of Short Trades,2.00
9,Max Gain,5.55


### Recipe 6: EMA-Regular-Order Strategy: Fetching Paper Trading Report - Order History

In [21]:
order_history = algobulls_connection.get_papertrading_report_order_history(strategy_code1)
print(order_history)



+-------------+---------------------+----------------------------------+------+
| INST        | TIME                | ID                               | TT   |
|-------------+---------------------+----------------------------------+------|
| NSE_EQ:SBIN | 2020-07-09 10:30:00 | 96c24ca4b3e448f381fc5c2bc52f7a29 | BUY  |
+-------------+---------------------+----------------------------------+------+
+----+---------------------+------------------------+-------+
|    | TIME                | STATE                  | MSG   |
|----+---------------------+------------------------+-------|
|  0 | 2020-07-09 10:30:00 | PUT ORDER REQ RECEIVED |       |
|  1 | 2020-07-09 10:30:00 | VALIDATION PENDING     |       |
|  2 | 2020-07-09 10:30:00 | OPEN PENDING           |       |
|  3 | 2020-07-09 10:30:00 | OPEN                   |       |
|  4 | 2020-07-09 10:30:00 | COMPLETE               |       |
+----+---------------------+------------------------+-------+

+-------------+---------------------+--

### Recipe 7: MACD-Bracket-Order Strategy: Fetching the Strategy

In [22]:
from pyalgotrading.algobulls import AlgoBullsConnection

In [23]:
algobulls_connection = AlgoBullsConnection()

In [24]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [25]:
algobulls_connection.set_access_token('80b7a69b168c5b3f15d56688841a8f2da5e2ab2c')

In [26]:
all_strategies = algobulls_connection.get_all_strategies()
all_strategies

,strategyCode,strategyName
0,49287246f9704bbcbad76ade9e2091d9,EMA Regular Order Strategy
1,4faf514fe096432b8e9f80f5951bd2ea,MACD Bracket Order Strategy


In [27]:
strategy_code2 = all_strategies.iloc[1]['strategyCode']
strategy_code2

'4faf514fe096432b8e9f80f5951bd2ea'

In [28]:
strategy_details2 = algobulls_connection.get_strategy_details(strategy_code2)
print(strategy_details2)

class StrategyMACDBracketOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.fastMA_period = self.strategy_parameters['fastma_period']
        self.slowMA_period = self.strategy_parameters['slowma_period']
        self.signal_period = self.strategy_parameters['signal_period']
        self.stoploss = self.strategy_parameters['stoploss_trigger']
        self.target = self.strategy_parameters['target_trigger']
        self.trailing_stoploss = self.strategy_parameters['trailing_stoploss_trigger']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'MACD Bracket Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        macdline, macdsignal, _ = talib.MACD(hist_data['close

### Recipe 8: MACD-Bracket-Order Strategy: Paper Trading the Strategy

In [29]:
from datetime import time
from pyalgotrading.constants import *

In [30]:
instruments = algobulls_connection.search_instrument('TATASTEEL')
instruments

[{'id': 1, 'value': 'NSE:TATASTEEL'}]

In [31]:
instrument = instruments[0]['value']
instrument

'NSE:TATASTEEL'

In [32]:
algobulls_connection.papertrade(strategy_code=strategy_code2, 
                                start_time=time(hour=9, minute=15), 
                                end_time=time(hour=15, minute=30), 
                                instrument=instrument,
                                lots=1,
                                strategy_parameters={
                                    'fastma_period': 26, 
                                    'slowma_period': 6, 
                                    'signal_period': 9,
                                    'target_trigger': 0.01, 
                                    'stoploss_trigger': 0.01, 
                                    'trailing_stoploss_trigger': 1
                                },
                                candle_interval=CandleInterval.MINUTES_15)

Setting Strategy Config... Success.
Submitting PAPERTRADING job... Success.


In [33]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTING'}

In [34]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STARTED'}

### There's more...

In [35]:
algobulls_connection.stop_papertrading_job(strategy_code2)

Stopping PAPERTRADING job... Success.


In [36]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STOPPING'}

In [37]:
algobulls_connection.get_papertrading_job_status(strategy_code2)

{'data': 'STOPPED'}

<img src="images/diagrams/abc-job-status-state-diagram.png" width='50%'>

### Recipe 9: MACD-Bracket-Order Strategy: Fetching Paper Trading Logs in Realtime

In [38]:
logs = algobulls_connection.get_papertrading_logs(strategy_code2)
print(logs)

[2020-07-09 09:14:12] Logs not available yet. Please retry in sometime.


In [39]:
logs = algobulls_connection.get_papertrading_logs(strategy_code2)
print(logs)

[2020-07-09 09:14:32] Performing sanity checks on cfg strategy_parameters, setting up broker connection and required data structures...
[2020-07-09 09:14:32] ABBroker connection has been setup successfully.
[2020-07-09 09:14:32] Sanity checks on cfg successful.
[2020-07-09 09:14:32] Setting up broker connection...
[2020-07-09 09:14:32] Broker connection has been setup successfully.
[2020-07-09 09:14:32] (NSE_EQ) Funds available in client's ABVirtualBroker account is : Rs. '1000000000.00'
[2020-07-09 09:14:32] 
########################################
 INITIALIZING ALGOBULLS CORE (v3.2.0)... 
########################################
[2020-07-09 09:14:32] Welcome ALGOBULLS VIRTUAL USER!
[2020-07-09 09:14:32] Reading strategy...
[2020-07-09 09:14:32] Entering Paper Trading mode. Henceforth, all timestamps will be Paper Trading timestamps...
[PT] [2020-07-09 09:15:00] [INFO] [tls] STARTING ALGOBULLS CORE...
[PT] [2020-07-09 09:15:00] [INFO] [tls] 
            
  #####  #######    #    ####

### Recipe 10: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Profit-&-Loss Table

In [40]:
algobulls_connection.get_papertrading_report_pnl_table(strategy_code2)

,instrument,entry_timestamp,entry_transaction_type,entry_quantity,entry_price,exit_time,exit_transaction_type,exit_quantity,exit_price,pnl_absolute,pnl_percentage,pnl_cumulative_absolute,pnl_cumulative_percentage
0,NSE_EQ:TATASTEEL,2020-07-09 12:00:00,SELL,1,345.2,2020-07-09 15:30:00,BUY,1,345,0.2,0.06,3.65,1.06
1,NSE_EQ:TATASTEEL,2020-07-09 09:45:00,BUY,1,345,2020-07-09 10:00:00,SELL,1,348.45,3.45,1,3.45,1


### Recipe 11: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Statistics Table

In [41]:
algobulls_connection.get_papertrading_report_statistics(strategy_code2)

,highlight_type,highlight_value
0,Net PnL,3.65
1,Net PnL %,1.06
2,Max Drawdown,3.45
3,Max Drawdown %,1
4,Number of Trades,2
5,Number of Wins,2
6,Number of Looses,0
7,Number of Long Trades,1
8,Number of Short Trades,1
9,Max Gain,3.45


### Recipe 12: MACD-Bracket-Order Strategy: Fetching Paper Trading Report - Order History

In [42]:
order_history = algobulls_connection.get_papertrading_report_order_history(strategy_code2)
print(order_history)



+------------------+---------------------+----------------------------------+------+
| INST             | TIME                | ID                               | TT   |
|------------------+---------------------+----------------------------------+------|
| NSE_EQ:TATASTEEL | 2020-07-09 09:45:00 | a310755e3d8b4a1ab4667882bf25751d | BUY  |
+------------------+---------------------+----------------------------------+------+
+----+---------------------+------------------------+-------+
|    | TIME                | STATE                  | MSG   |
|----+---------------------+------------------------+-------|
|  0 | 2020-07-09 09:45:00 | PUT ORDER REQ RECEIVED |       |
|  1 | 2020-07-09 09:45:00 | VALIDATION PENDING     |       |
|  2 | 2020-07-09 09:45:00 | OPEN PENDING           |       |
|  3 | 2020-07-09 09:45:00 | OPEN                   |       |
|  4 | 2020-07-09 09:45:00 | COMPLETE               |       |
+----+---------------------+------------------------+-------+

+-------------